Makes Table 1 & 2 and takes all of the preliminary SNeCO data and organizes it, preparing it for when we begin making intensity measurements in section 3.

To run this notebook we need '2.SNeCOMasterCatManualClean.txt' (see prepwork below). Once this prepwork is done, it is not needed to repeat unless we update the sample / redownload the OSC.

We also need "0.PHANGS_v1p6.csv" to pull other galaxy information from the survey. 
We also use "0.PHANGSMapFiles.txt" to pull the telescope orientations used for each galaxy.

      *PREPWORK* 
      
After looking at the galaxy images from #2. We find we need to remove one SN that isnt within the region covered by the PHANGS-ALMA footprint. We remove this (SN2003gd from NGC0628) and also the 6 candidate SNe, the one duplicate entry, and the LRN, and begin to perform some rudimentary stats in small tables.

Before we run this code we do the following prep:

We take the 2.SNeCOMasterCat.txt file and manually remove the following SNe:

Previously (& Permanently) Removed
AT2020nqq   (LRN) 
PTSS-19clju (Duplicate)

Recently commented out
AT2018eoq   (Candidate)
AT2019pck   (Candidate)
AT2019npi   (Candidate)
AT2019npd   (Candidate)
AT2020hol   (Candidate)
AT2020juh   (Candidate)
AT2020cwh   (Candidate)
SN2003gd    (Out of Map ngc0628)
SN2005af    (Out of Map ngc4945)
ngc5236,PS  (Not in Map & Also not good data on this one)
SN2019qyl (IIn/LBV)
SN2019krl (IIn/LBV)
SN1997bs (IIn)

Then we save this as 2.SNeCOMasterCatManualClean.txt

Should now have 63 SNe.

We turn this file into a dataframe and use to make basic tables of stats.
**********************
    
    *MORE PREPWORK* Changed 'name' column to 'galaxy' & 'eso097-013' to 'circinus' in 0.PHANGS_v1p6.csv file
    

PHANGS SURVEY FILE 0.PHANGS_v1p6.csv file has following columns:

### galaxy identifiers
galaxy,pgc,alias,

### survey information
survey_astrosat_status,survey_astrosat_instrument,survey_astrosat_notes,survey_astrosat_references,survey_galex_status,survey_galex_instrument,survey_galex_notes,survey_galex_references,survey_halpha_status,survey_halpha_instrument,survey_halpha_notes,survey_halpha_references,survey_muse_status,survey_muse_instrument,survey_muse_notes,survey_muse_references,survey_kcwi_status,survey_kcwi_instrument,survey_kcwi_notes,survey_kcwi_references,survey_sitelle_status,survey_sitelle_instrument,survey_sitelle_notes,survey_sitelle_references,survey_typhoon_status,survey_typhoon_instrument,survey_typhoon_notes,survey_typhoon_references,survey_hst_status,survey_hst_instrument,survey_hst_notes,survey_hst_references,survey_irac_status,survey_irac_instrument,survey_irac_notes,survey_irac_references,survey_mips_status,survey_mips_instrument,survey_mips_notes,survey_mips_references,survey_wise_status,survey_wise_instrument,survey_wise_notes,survey_wise_references,survey_herschel_status,survey_herschel_instrument,survey_herschel_notes,survey_herschel_references,survey_alma_status,survey_alma_instrument,survey_alma_notes,survey_alma_references,survey_otherco_status,survey_otherco_instrument,survey_otherco_notes,survey_otherco_references,survey_hiresco_status,survey_hiresco_instrument,survey_hiresco_notes,survey_hiresco_references,survey_multijco_status,survey_multijco_instrument,survey_multijco_notes,survey_multijco_references,survey_dense_status,survey_dense_instrument,survey_dense_notes,survey_dense_references,survey_hi_status,survey_hi_instrument,survey_hi_notes,survey_hi_references,

### orientation
orient_ra,orient_ra_unc,orient_dec,orient_dec_unc,orient_pos_ref,orient_vlsr,orient_vlsr_unc,orient_vlsr_ref,orient_incl,orient_incl_unc,orient_posang,orient_posang_unc,orient_ref,

### rotation
rotcur_v0,rotcur_v0_unc,rotcur_rt,rotcur_rt_unc,rotcur_ref,

### distance
dist,dist_unc,dist_label,dist_ref,

### size properties
size_r90,size_r90_unc,size_r90_ref,mwext_sfd98,mwext_sfd98_unc,mwext_sf11,mwext_sf11_unc,z0mgs_mtol,props_mtol_method,props_sfr,props_sfr_unc,props_sfr_method,props_mstar,props_mstar_unc,props_mtol,props_deltams,props_deltams_unc,props_mhi,props_mhi_unc,lco_phangs,lco_phangs_unc,appcor_phangs,aco10_phangs,size_r25,size_r25_unc,size_reff,size_reff_unc,size_scalelength,size_scalelength_unc,mh2_phangs,mh2_phangs_unc,

### morphology
morph_t,morph_t_unc,morph_string,morph_bar,morph_buta15,morph_class,morph_gold,morph_bar_flag,morph_bar_axis_ratio,morph_bar_pa,morph_bar_r,morph_bar_red,morph_bar_q_flag,morph_nuclear_bar_axis_ratio,morph_nuclear_bar_pa,morph_nuclear_bar_r,morph_spiral_arms,morph_multi_arm,morph_featureless,morph_disk_1_axis_ratio,morph_disk_1_pa,morph_disk_1_r,morph_disk_2_axis_ratio,morph_disk_2_pa,morph_disk_2_r,morph_bulge_axis_ratio,morph_bulge_pa,morph_bulge_r,morph_rings_outer,morph_ring_inner,morph_lens_1_axis_ratio,morph_lens_1_pa,morph_lens_1_r,morph_lens_2_axis_ratio,morph_lens_2_pa,morph_lens_2_r,morph_ring_1_axis_ratio,morph_ring_1_pa,morph_ring_1_r,morph_ring_2_axis_ratio,morph_ring_2_pa,morph_ring_2_r,morph_vcc,morph_interacting,morph_in_virgo_fornax,morph_high_peculiar,morph_comments,

### agn
agn_veron_y_n,agn_veron_class,agn_milliquas_y_n,agn_milliquas_class_pQSO

In [10]:
# IMPORT PACKAGES
import numpy as np
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table, join
from astropy.wcs import WCS
from astropy.io import ascii
import astropy
from astropy.io import fits



In [11]:
# Prep table 1 - OSC Objects

OSC_File = '../Data/2.SNeCOMasterCat.txt'
OSC_Objects = Table.read(OSC_File, format='csv') 

Clean_File = '../Data/2.SNeCOMasterCatManualClean.txt'
SNeSample = Table.read(Clean_File, format='csv')

SNeSample

Galaxy,SNName,SNType,ra,Dec,xCoord,yCoord,co21int,co21err,co21ew,beamsize
str8,str11,str12,float64,float64,float64,float64,float64,float64,float64,float64
circinus,SN1996cr,II,213.2918,-65.3457,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078
ngc0253,SN1940E,I,11.8783,-25.2934,296.9010534051223,236.26662905273994,219.30367,1.465811,45.862267,0.00232543637022
ngc0628,SN2013ej,II,24.2007,15.7586,333.46022572413716,356.77980642079694,5.7713037,1.9949342,nan,0.0003114595189268
ngc1068,SN2018ivc,II,40.672,-0.0088,41.400000046852824,53.87249997022789,127.77749,0.6721418,32.0753,0.002414131938401
ngc1087,SN1995V,II,41.6115,-0.4988,161.4198245203906,287.6499560537321,11.513839,1.1399242,5.175495,0.0004451676280474
ngc1097,SN1999eu,II,41.5866,-30.3184,245.2470996170838,7.223039949207134,0.0,0.7541714,nan,0.0004711325009833
ngc1097,SN1992bd,II,41.5792,-30.2756,291.5099030167368,315.426803230518,296.15988,2.0283754,45.82589,0.0004711325009833
ngc1365,SN2012fr,Ia,53.4006,-36.1268,390.2906744443858,389.8499395285312,-2.3150725,1.1174963,nan,0.0003829740084505
ngc1365,SN2001du,II,53.3713,-36.1421,673.9024849664049,205.70404673775522,3.0703216,1.3357118,nan,0.0003829740084505


In [12]:

ra, dec, papers, flags = [],[],[],[]

for i in range(len(OSC_Objects['#Galaxy'])):
    
    ra.append(round(OSC_Objects["Ra"][i],4))
    dec.append(round(OSC_Objects["Dec"][i],4))
    papers.append('?')
    
    if OSC_Objects['Supernova'][i] in SNeSample['SNName']:
        flags.append('\checkmark')
    
    else: flags.append('-')
                     
OSCtable = Table({'Supernova'            : OSC_Objects['Supernova'],
                       'Type'            : OSC_Objects['Type'],
                       'Galaxy'          : OSC_Objects['#Galaxy'],
                       'Ra'              : ra,
                       'Dec'             : dec,
                       'Reference Paper' : papers,            
                       'SNeCOSample'     : flags})

ascii.write(OSCtable, '../Data/3.OSCTable.tex', format='latex', fast_writer=False, overwrite=True)


In [13]:
OSCtable

Supernova,Type,Galaxy,Ra,Dec,Reference Paper,SNeCOSample
str176,str12,str8,float64,float64,str1,str10
AT2020juh,Candidate,circinus,213.3379,-65.3413,?,-
SN1996cr,II,circinus,213.2918,-65.3457,?,\checkmark
AT2019npi,Candidate,ngc0247,11.7453,-20.7083,?,-
AT2021wtu,Candidate,ngc0247,11.7842,-20.7788,?,-
AT2019npd,Candidate,ngc0253,11.7363,-25.3768,?,-
AT2019pck,Candidate,ngc0253,11.8514,-25.2261,?,-
AT2020hol,Candidate,ngc0253,11.8539,-25.3571,?,-
SN1940E,I,ngc0253,11.8783,-25.2934,?,\checkmark
SN2019qyl,IIn/LBV,ngc0300,13.7399,-37.6444,?,-


In [14]:
# Turn file into dataStructure and clean

dataFile      = '../Data/2.SNeCOMasterCatManualClean.txt'
galaxy, SNname, SNtype = np.genfromtxt(dataFile,  dtype=str, unpack=True, delimiter = ',', usecols=(0,1,2))
ra, dec, xVal, yVal, co_21_int, co_21_err, co_21_ew, beamsize  = np.genfromtxt(dataFile,  dtype=float, unpack=True, delimiter = ',', usecols=(3,4,5,6,7,8,9,10))

# remove spaces and " " from SN data
galaxy = [x.strip() for x in galaxy]
#SNname = [x[2:-2] for x in SNname]
SNtype = [x.strip() for x in SNtype]

# build table 
data = Table({'galaxy'       : galaxy,
              'SN_name'      : SNname,
              'SN_type'      : SNtype,
              'SN_ra'        : ra,
              'SN_dec'       : dec,
              'SN_xCoord'    : xVal,
              'SN_yCoord'    : yVal,
              'SN_co21int'   : co_21_int,
              'SN_co21err'   : co_21_err,
              'SN_co21ew'    : co_21_ew,
              'map_beamsize' : beamsize,
             },
             names=['galaxy', 'SN_name','SN_type','SN_ra','SN_dec','SN_xCoord', 'SN_yCoord', 'SN_co21int', 'SN_co21err', "SN_co21ew", 'map_beamsize'])


In [20]:
# Pull Galaxy data from PHANGS survey

PHANGS_data = '../Data/0.PHANGS_v1p6.csv'
PHANGSTable = Table.read(PHANGS_data, format='csv')

PHANGS_clean = PHANGSTable['galaxy', 'orient_ra', 'orient_dec', 'dist', 'orient_posang', 'orient_incl', 
                           'size_r25','props_mstar','props_sfr','morph_t','morph_bar','survey_alma_status']

In [55]:
PHANGS_clean

galaxy,orient_ra,orient_dec,dist,orient_posang,orient_incl,size_r25,props_mstar,props_sfr,morph_t,morph_bar,survey_alma_status
str8,float64,float64,float64,float64,float64,float64,float64,float64,float64,str3,str21
circinus,213.29124,-65.33915,4.2,36.74,64.3,262.49515,33909813000.0,4.0878983,3.299999952316284,--,released
ic0010,5.09625,59.2931,0.79,31.1,129.0,200.5032,nan,nan,9.899999618530273,BAR,cannot_be_observed
ic0342,56.702126,68.09609,3.45,42.0,31.0,593.0909,17468820000.0,4.3485565,6.0,BAR,cannot_be_observed
ic1954,52.879707,-51.90486,12.8,63.4,57.1,89.76793,4665490400.0,0.36467403,3.299999952316284,BAR,released
ic1993,56.769936,-33.70967,18.09,353.5,21.3,84.357025,10645845000.0,0.18425666,3.0,BAR,not_in_survey
ic5273,344.86118,-37.70284,14.18,234.1,52.0,91.858894,5276273000.0,0.53830403,5.599999904632568,BAR,released
ic5332,353.61453,-36.10108,9.01,74.4,26.9,182.0209,4655067600.0,0.41174614,6.800000190734863,BAR,released
ngc0224,10.684684,41.26898,0.82,38.0,77.7,5334.8384,nan,nan,3.0,--,cannot_be_observed
ngc0247,11.785718,-20.76045,3.71,167.44,76.38,590.3658,3258368800.0,0.1779774,6.900000095367432,BAR,released


In [52]:
allGalMass, allGalSFR, allGalMorphT, allGalMorphBar = [],[],[],[]
sampleMass, sampleSFR, sampleMorphT, sampleMorphBar = [],[],[],[]
noSamMass, noSamSFR, noSamMorphT, noSamMorphBar = [],[],[],[]

unique, noSamUnique = [],[]
numBar=0
numSampleBar=0
numNoSamBar = 0


for i in range(len(PHANGS_clean)):
    if(PHANGS_clean['galaxy'][i] == 'ngc1068'): 
        unique.append('ngc1068')
        allGalMass.append(PHANGS_clean['props_mstar'][i])
        allGalSFR.append(PHANGS_clean['props_sfr'][i])
        allGalMorphT.append(PHANGS_clean['morph_t'][i])
        allGalMorphBar.append(PHANGS_clean['morph_bar'][i]) 
        if PHANGS_clean['morph_bar'][i] == 'BAR':
            numBar += 1
        sampleMass.append(PHANGS_clean['props_mstar'][i])
        sampleSFR.append(PHANGS_clean['props_sfr'][i])
        sampleMorphT.append(PHANGS_clean['morph_t'][i])
        sampleMorphBar.append(PHANGS_clean['morph_bar'][i]) 
        if PHANGS_clean['morph_bar'][i] == 'BAR':
            numSampleBar += 1
    
    if PHANGS_clean['survey_alma_status'][i] == 'released':
        allGalMass.append(PHANGS_clean['props_mstar'][i])
        allGalSFR.append(PHANGS_clean['props_sfr'][i])
        allGalMorphT.append(PHANGS_clean['morph_t'][i])
        allGalMorphBar.append(PHANGS_clean['morph_bar'][i]) 
        if PHANGS_clean['morph_bar'][i] == 'BAR':
            numBar += 1
            
        if(PHANGS_clean['galaxy'][i] == 'ngc0300'): 
            unique.append('ngc0300')
            sampleMass.append(PHANGS_clean['props_mstar'][i])
            sampleSFR.append(PHANGS_clean['props_sfr'][i])
            sampleMorphT.append(PHANGS_clean['morph_t'][i])
            sampleMorphBar.append(PHANGS_clean['morph_bar'][i]) 
            if PHANGS_clean['morph_bar'][i] == 'BAR':
                numSampleBar += 1


            
        for j in range(len(data['galaxy'])):

            if(PHANGS_clean['galaxy'][i] == data['galaxy'][j]) and data['galaxy'][j] not in unique:
                unique.append(data['galaxy'][j])
                sampleMass.append(PHANGS_clean['props_mstar'][i])
                sampleSFR.append(PHANGS_clean['props_sfr'][i])
                sampleMorphT.append(PHANGS_clean['morph_t'][i])
                sampleMorphBar.append(PHANGS_clean['morph_bar'][i]) 
                if PHANGS_clean['morph_bar'][i] == 'BAR':
                    numSampleBar += 1

        if(PHANGS_clean['galaxy'][i] not in data['galaxy']) and PHANGS_clean['galaxy'][i] not in noSamUnique and PHANGS_clean['galaxy'][i]!="ngc0300":
            noSamUnique.append(PHANGS_clean['galaxy'][i])
            noSamMass.append(PHANGS_clean['props_mstar'][i])
            noSamSFR.append(PHANGS_clean['props_sfr'][i])
            noSamMorphT.append(PHANGS_clean['morph_t'][i])
            noSamMorphBar.append(PHANGS_clean['morph_bar'][i]) 
            if PHANGS_clean['morph_bar'][i] == 'BAR':
                numNoSamBar += 1


print(len(allGalMass), len(sampleMass), len(noSamMass))
print(numBar/len(allGalMorphBar), numSampleBar/len(sampleMorphBar), numNoSamBar/len(noSamMorphBar))
print(unique)
print(noSamUnique)

89 31 58
0.7752808988764045 0.7096774193548387 0.8103448275862069
['circinus', 'ngc0253', 'ngc0300', 'ngc0628', 'ngc1068', 'ngc1087', 'ngc1097', 'ngc1365', 'ngc1433', 'ngc1511', 'ngc1559', 'ngc1566', 'ngc1637', 'ngc1672', 'ngc2775', 'ngc2997', 'ngc3239', 'ngc3627', 'ngc4254', 'ngc4303', 'ngc4321', 'ngc4424', 'ngc4457', 'ngc4496a', 'ngc4536', 'ngc4579', 'ngc4945', 'ngc5128', 'ngc5530', 'ngc6744', 'ngc7793']
['ic1954', 'ic5273', 'ic5332', 'ngc0247', 'ngc0685', 'ngc1313', 'ngc1300', 'ngc1317', 'ngc1385', 'ngc1512', 'ngc1546', 'ngc1809', 'ngc1792', 'ngc2090', 'ngc2283', 'ngc2566', 'ngc2835', 'ngc2903', 'ngc3059', 'ngc3137', 'ngc3351', 'ngc3489', 'ngc3511', 'ngc3507', 'ngc3521', 'ngc3596', 'ngc3599', 'ngc3621', 'ngc3626', 'ngc4207', 'ngc4293', 'ngc4298', 'ngc4459', 'ngc4476', 'ngc4477', 'ngc4535', 'ngc4540', 'ngc4548', 'ngc4569', 'ngc4571', 'ngc4596', 'ngc4654', 'ngc4689', 'ngc4694', 'ngc4731', 'ngc4781', 'ngc4826', 'ngc4941', 'ngc4951', 'ngc5042', 'ngc5068', 'ngc5134', 'ngc5248', 'ngc5643'

In [62]:
medAllGalMass = np.nanmean(np.log10(allGalMass))
medSampleMass = np.nanmean(np.log10(sampleMass))
medNoSamMass = np.nanmean(np.log10(noSamMass))

medAllGalSFR = np.nanmean(np.log10(allGalSFR))
medSampleSFR = np.nanmean(np.log10(sampleSFR))
medNoSamSFR = np.nanmean(np.log10(noSamSFR))

medAllGalMorphT = np.nanmean(allGalMorphT)
medSampleMorphT = np.nanmean(sampleMorphT)
medNoSamMorphT = np.nanmean(noSamMorphT)


print(medAllGalMass, medSampleMass, medNoSamMass)
print(medAllGalSFR, medSampleSFR, medNoSamSFR)
print(medAllGalMorphT, medSampleMorphT, medNoSamMorphT)

10.240933980017322 10.39854782086653 10.156692099563434
-0.05829490736923828 0.27322854668311686 -0.2354884776386005
3.7280898794029538 4.045161276094375 3.5586206846196076


In [58]:
print(allGalMorphT)
print()
print(np.sort(sampleMorphT))
print()
print(np.sort(noSamMorphT))


[3.299999952316284, 3.299999952316284, 5.599999904632568, 6.800000190734863, 6.900000095367432, 5.099999904632568, 6.900000095367432, 5.199999809265137, 5.400000095367432, 3.0, 5.199999809265137, 3.299999952316284, 7.0, 4.0, 0.800000011920929, 3.200000047683716, 5.900000095367432, 1.5, 2.0, 1.2000000476837158, -0.4000000059604645, 5.900000095367432, 4.0, 5.0, 3.299999952316284, 5.0, 4.0, 4.5, 5.900000095367432, 2.700000047683716, 1.600000023841858, 5.0, 4.0, 5.099999904632568, 4.0, 5.900000095367432, 9.800000190734863, 3.0999999046325684, -1.2000000476837158, 5.099999904632568, 3.0999999046325684, 4.0, 5.199999809265137, -2.0, 6.900000095367432, -0.800000011920929, 3.0999999046325684, 7.699999809265137, 5.199999809265137, 0.30000001192092896, 5.099999904632568, 4.0, 4.0, 1.2999999523162842, 0.30000001192092896, -1.600000023841858, -2.9000000953674316, -1.7000000476837158, 7.400000095367432, 5.0, 4.300000190734863, 6.199999809265137, 3.0999999046325684, 2.4000000953674316, 6.40000009536

In [61]:
aveAllGalMorphT = np.average(allGalMorphT)
aveSampleMorphT = np.average(sampleMorphT)
aveNoSamMorphT = np.average(noSamMorphT)
print(aveAllGalMorphT,aveSampleMorphT, aveNoSamMorphT)

3.7280898794029538 4.045161276094375 3.5586206846196076


In [ ]:
for i in range(len(PHANGSTable)):
    print(PHANGSTable['galaxy'][i], PHANGSTable['survey_alma_instrument'][i])


In [8]:
SN_plus_gal_data = join(data, PHANGS_clean, keys='galaxy')

SN_plus_gal_data.colnames

# for i in range(len(SN_plus_gal_data)):
#     print(SN_plus_gal_data['galaxy'][i],)

['galaxy',
 'SN_name',
 'SN_type',
 'SN_ra',
 'SN_dec',
 'SN_xCoord',
 'SN_yCoord',
 'SN_co21int',
 'SN_co21err',
 'SN_co21ew',
 'map_beamsize',
 'orient_ra',
 'orient_dec',
 'dist',
 'orient_posang',
 'orient_incl',
 'size_r25']

In [9]:
# Pull Telescope Orientation data from PHANGSMapFiles.csv and join to table

telOrientFile = '../Data/0.PHANGSMapFiles.csv'
telOrientTable = Table.read(telOrientFile, format='csv')

ImageFiles150, ErrFiles150, AlphaCOFiles = [],[],[]

for i in range(len(telOrientTable)):
    
    galaxy = telOrientTable['galaxy'][i]
    GalName = galaxy.upper()
    
    if GalName == 'CIRCINUS':
        GalName = 'ESO097-013'
    
    ImageFiles150.append("/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/" + galaxy + "/" + galaxy + "_" + telOrientTable['telOrient'][i] + "_co21_150pc_broad_mom0.fits")
    ErrFiles150.append("/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/" + galaxy + "/" + galaxy + "_" + telOrientTable['telOrient'][i] + "_co21_150pc_broad_emom0.fits")
    AlphaCOFiles.append("/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/" + GalName + '_alphaCO21_scaling.fits')

telOrientTable.add_column(ImageFiles150, name = "ImageFile150")
telOrientTable.add_column(ErrFiles150, name = "ErrFile150")
telOrientTable.add_column(AlphaCOFiles, name = "AlphaCOFile")   
    

In [10]:
for i in range(len(telOrientTable)):
    print(telOrientTable[i])

 galaxy  telOrient                                                 ImageFile150                                                                                                   ErrFile150                                                                                    AlphaCOFile                                   
-------- --------- ------------------------------------------------------------------------------------------------------------ ------------------------------------------------------------------------------------------------------------- --------------------------------------------------------------------------------
circinus     7m+tp /data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_mom0.fits /data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_emom0.fits /data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/ESO097-013_alphaCO21_scaling.fits
galaxy telOrient                           

In [11]:
SN_plus_gal_plus_telOrient_data = join(SN_plus_gal_data,telOrientTable, keys='galaxy')


In [12]:
SN_plus_gal_plus_telOrient_data

galaxy,SN_name,SN_type,SN_ra,SN_dec,SN_xCoord,SN_yCoord,SN_co21int,SN_co21err,SN_co21ew,map_beamsize,orient_ra,orient_dec,dist,orient_posang,orient_incl,size_r25,telOrient,ImageFile150,ErrFile150,AlphaCOFile
str8,str12,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,str112,str113,str80
circinus,SN1996cr,II,213.2918,-65.3457,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078,213.29124,-65.33915,4.2,36.74,64.3,262.49515,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/circinus/circinus_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/ESO097-013_alphaCO21_scaling.fits
ngc0253,SN1940E,I,11.8783,-25.2934,296.9010534051223,236.26662905273994,219.30367,1.465811,45.862267,0.00232543637022,11.887966,-25.288443,3.7,52.48,75.0,803.7504,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0253/ngc0253_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0253/ngc0253_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/NGC0253_alphaCO21_scaling.fits
ngc0628,SN2013ej,II,24.2007,15.7586,333.46022572413716,356.77980642079694,5.7713037,1.9949342,nan,0.0003114595189268,24.173855,15.783643,9.84,20.7,8.9,296.56592,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0628/ngc0628_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc0628/ngc0628_12m+7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/NGC0628_alphaCO21_scaling.fits
ngc1068,SN2018ivc,II,40.672,-0.0088,41.400000046852824,53.87249997022789,127.77749,0.6721418,32.0753,0.002414131938401,40.66973,-0.0132779,13.97,72.65,34.7,183.28261,7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1068/ngc1068_7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1068/ngc1068_7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/NGC1068_alphaCO21_scaling.fits
ngc1087,SN1995V,II,41.6115,-0.4988,161.4198245203906,287.6499560537321,11.513839,1.1399242,5.175495,0.0004451676280474,41.60492,-0.498717,15.85,359.1,42.9,89.14999,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1087/ngc1087_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1087/ngc1087_12m+7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/NGC1087_alphaCO21_scaling.fits
ngc1097,SN1999eu,II,41.5866,-30.3184,245.2470996170838,7.223039949207134,0.0,0.7541714,nan,0.0004711325009833,41.578957,-30.274675,13.58,122.4,48.6,317.0452,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1097/ngc1097_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1097/ngc1097_12m+7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/NGC1097_alphaCO21_scaling.fits
ngc1097,SN1992bd,II,41.5792,-30.2756,291.5099030167368,315.426803230518,296.15988,2.0283754,45.82589,0.0004711325009833,41.578957,-30.274675,13.58,122.4,48.6,317.0452,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1097/ngc1097_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1097/ngc1097_12m+7m+tp_co21_150pc_broad_emom0.fits,/data/kant/0/sun.1608/PHANGS/ALMA/alphaCO/v1p0/NGC1097_alphaCO21_scaling.fits
ngc1365,SN2001du,II,53.3713,-36.1421,673.9024849664049,205.70404673775522,3.0703216,1.3357118,nan,0.0003829740084505,53.40152,-36.140404,19.57,201.1,55.4,360.67926,12m+7m+tp,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc1365/ngc1365_12m+7m+tp_co21_150pc_broad_mom0.fits,/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc

In [13]:
ascii.write(SN_plus_gal_plus_telOrient_data, '../Data/3.SNe+GalData.csv', format='csv', fast_writer=False, overwrite=True)  

In [14]:
for i in range(len(SN_plus_gal_plus_telOrient_data)):
    print(SN_plus_gal_plus_telOrient_data['galaxy'][i])

circinus
ngc0253
ngc0628
ngc1068
ngc1087
ngc1097
ngc1097
ngc1365
ngc1365
ngc1365
ngc1433
ngc1511
ngc1559
ngc1559
ngc1559
ngc1559
ngc1566
ngc1566
ngc1637
ngc1672
ngc2775
ngc2997
ngc2997
ngc3239
ngc3627
ngc3627
ngc3627
ngc3627
ngc4254
ngc4254
ngc4254
ngc4254
ngc4303
ngc4303
ngc4303
ngc4303
ngc4303
ngc4303
ngc4303
ngc4321
ngc4321
ngc4321
ngc4321
ngc4321
ngc4321
ngc4424
ngc4457
ngc4496a
ngc4496a
ngc4536
ngc4579
ngc4579
ngc4945
ngc5128
ngc5128
ngc5236
ngc5236
ngc5236
ngc5236
ngc5236
ngc5530
ngc6744
ngc7793


In [29]:
# Calculating completeness for high and low mass galaxies
PA_completeness = Table.read("../Data/0.PHANGS-ALMA_completeness.txt", format="ascii")

lmCom, hmCom, allCom = [],[],[]

for i in range(len(PA_completeness)):
    if PA_completeness["completeness(150pc)"][i] != "-" and PA_completeness["completeness(150pc)"][i] != "0.9F":
        allCom.append(np.float(PA_completeness["completeness(150pc)"][i]))
        if PA_completeness['mass(log10_Msun)'][i]>=10:
            hmCom.append(np.float(PA_completeness["completeness(150pc)"][i]))
        else:
            lmCom.append(np.float(PA_completeness["completeness(150pc)"][i]))

<ipython-input-29-9c03378e36ef>:8: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  allCom.append(np.float(PA_completeness["completeness(150pc)"][i]))
<ipython-input-29-9c03378e36ef>:10: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  hmCom.append(np.float(PA_completeness["completeness(150pc)"][i]))
<ipython-input-29-9c03378e36ef>:12: Deprecat

In [30]:
print(np.median(hmCom))
print(np.median(lmCom))
print(np.median(allCom))

77.7
68.45
76.8


In [31]:
print(np.average(hmCom))
print(np.average(lmCom))
print(np.average(allCom))

74.62380952380951
60.48333333333333
71.48148148148147
